# Identity Verification Getting Started

-------

This lab simply checks to make sure that your environment is setup to run the identity verification labs. There are no-non standard libraries, however you want to make sure that boto3 is installed and updated. 

You can read more about boto3 here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html


### Check the Python Environment is Setup
-----
Make sure that the following blocks of code will run, make any changes to your region, bucket and clients necessary

In [ ]:
import boto3
import io
from IPython.display import display, Image as IImage
from PIL import Image, ImageDraw

%store -r bucket_name
mySession = boto3.session.Session()
aws_region = mySession.region_name
print("AWS Bucket: {}".format(bucket_name))
print("AWS Bucket: {}".format(aws_region))

### Setup Clients 
-----
Here we are going to use both S3 and Rekognition clients  

In [ ]:
s3_client  = boto3.client('s3')
rek_client = boto3.client('rekognition')

### Display a Face from S3
-------
Here we are going to display a face

In [ ]:
## Image of a Face
face_image = "face_1_0.jpeg"
print(face_image)
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : face_image})))

###  This function will list the files stored in our bucket. 
-----
We'll make use of these files in subsequent labs. 

In [ ]:
def list_s3_files_using_client(bucket_name):
    """
    This functions list all files in s3 bucket.
    :return: None
    """
    print(f"Files located in bucket: {bucket_name}")
    s3_client = boto3.client("s3")
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    files = response.get("Contents")
    for file in files:
        print(f"file_name: {file['Key']}, size: {file['Size']}")

# -- run our function 
list_s3_files_using_client(bucket_name)

## Check Rekognition APIs
-----
Here we'll use the detect_faces API to ensure that our Rekognition client is properly configured and we have the appropriate permissions. The following will produce a FaceDetails object. Take a moment to look at the result. 


In [ ]:
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket': bucket_name,
    'Name'  : face_image}},
    Attributes=['ALL'])

response

## Draw a Bounding Box around the detected Face
-----
Amazon Rekognition Image operations can return bounding boxes coordinates for items that are detected in images. For example, the DetectFaces operation returns a bounding box (BoundingBox) for each face detected in an image. You can use the bounding box coordinates to display a box around detected items. 

A BoundingBox has the following properties:

- Height – The height of the bounding box as a ratio of the overall image height.
- Left – The left coordinate of the bounding box as a ratio of overall image width.
- Top – The top coordinate of the bounding box as a ratio of overall image height.
- Width – The width of the bounding box as a ratio of the overall image width.

Each BoundingBox property has a value between 0 and 1. Each property value is a ratio of the overall image width (Left and Width) or height (Height and Top). For example, if the input image is 700 x 200 pixels, and the top-left coordinate of the bounding box is 350 x 50 pixels, the API returns a Left value of 0.5 (350/700) and a Top value of 0.25 (50/200).


In [ ]:
def show_faces(photo,bucket):
     
    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectFaces 
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        Attributes=['ALL'])

    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
                    
    # calculate and display bounding boxes for each detected face       
    print('Detected face for ' + photo)    
    for faceDetail in response['FaceDetails']:
        print('Predicted age between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
                

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )
        draw.line(points, fill='#00d400', width=4)
        
        display(image)


    return len(response['FaceDetails'])

response = show_faces(face_image,bucket_name)